- Haar : 좌표값이 나옴
- DNN : 비율이 나옴 - 사진 크기가 다 다르기 때문

In [1]:
import cv2
import numpy as np

In [2]:
# 사용할 모델 파일명
model_name = 'opencv_data/res10_300x300_ssd_iter_140000.caffemodel'
# 모델 내의 신경망 구조 정보 파일 명
prototxt_name = 'opencv_data/deploy.prototxt.txt'
# 얼굴이라고 인정할 최소 정확도
min_confidence = 0.3
# 사용할 파일 이름
file_name = 'opencv_data/image/marathon_01.jpg'

In [3]:
# 이미지를 불러온다
frame = cv2.imread(file_name)

In [4]:
# 정보를 확인한다.
print(f'width : {frame.shape[1]}')
print(f'height : {frame.shape[0]}')
print(f'channel : {frame.shape[2]}')

# 원본 이미지의 가로 세로 길이
width = frame.shape[1]
height = frame.shape[0]

width : 1024
height : 799
channel : 3


In [5]:
cv2.imshow('Original Image', frame)

In [6]:
# 얼굴 인식 작업
# 사용할 모델을 불러온다
model = cv2.dnn.readNetFromCaffe(prototxt_name, model_name)
# prototxt 파일에 이미지가 300x300, 3채널로 설정되어 있으므로 300x300 사이즈로 변환한다
a1 = cv2.resize(frame, (300, 300))
# cv2.imshow('a1', a1)
# 이진데이터로 변환한다
# 원본 이미지 데이터, 스케일링(크기조정), 결과데이터 행렬 사이즈, 표준화를 위해 각 색상에서 빼줄 색상값
blob = cv2.dnn.blobFromImage(a1, 1.0, (300, 300), (104.0, 177.0, 123.0))
# blob
model.setInput(blob)
# 얼굴 부분을 인식한다
detections = model.forward()
# 세번째 값이 얼굴이라고 인식된 부분의 개수
print(detections.shape)
# 얼굴이라고 인식된 부분의 개수만큼 반복한다
for i in range(0, detections.shape[2]):
    # 얼굴이라고 인지한 예측 정확도 추출
    confidence = detections[0, 0, i, 2]
    # print(confidence)
    
    # 정해놓은 최소 정확도 보다 높은 것만 사용한다
    if confidence > min_confidence:
    
        # 얼굴이라고 인식된 영역의 데이터를 추출한다
        box1 = detections[0, 0, i, 3:7]
        # print(box1)
        # 원본이미지에 맞게끔 좌표 계산을 해 준다
        box2 = box1 * np.array([width, height, width, height])
        print(box2)
        # 좌표를 정수 형태로 변환한다
        x1, y1, x2, y2 = box2.astype('int')
        # print(x1, y1, x2, y2)
        # 네모를 그린다
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # 예측 정확도를 표시한다
        text = f'{int(confidence * 100)}%'
        cv2.putText(frame, text, (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

cv2.imshow('detection', frame)

(1, 1, 200, 7)
[822.28503418 218.2037954  887.86462402 295.83666864]
[ 27.54515839 230.08258221  84.29226685 300.18741694]
[776.84698486 294.05883676 833.06207275 373.33155102]
[691.08520508 209.68462771 742.95837402 276.6282692 ]
[669.22998047 305.32267722 717.23803711 373.96168944]
[412.77496338 260.0827243  462.96118164 335.13727579]
[307.92382812 188.1953429  360.52825928 252.30825484]
[455.58612061 213.18204719 512.41314697 281.71038097]


In [7]:
cv2.waitKey()
cv2.destroyAllWindows()